In [ ]:
import random
import os
from os import listdir

import seaborn as sns

import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as F

from ipywidgets import interact, widgets

DATA_PATH = './data'
DATA_FILE = './data.zip'
KUDU_MASTER = 'kudu-master-1:7051'

# Kudu Tests

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.kudu:kudu-spark3_2.12:1.13.0.7.1.5.17-1 org.neo4j:neo4j-connector-apache-spark_2.12:5.0.1_for_spark_3 --repositories https://repository.cloudera.com/artifactory/cloudera-repos/ pyspark-shell'

In [ ]:
spark = SparkSession.builder.config('spark.packages', 'org.apache.kudu:kudu-spark3_2.12:1.13.0.7.1.5.17-1', 'org.neo4j:neo4j-connector-apache-spark_2.12:5.0.1_for_spark_3').getOrCreate()
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

# Neo4j Test

In [ ]:
df = spark.read.format("org.neo4j.spark.DataSource")\
 .option("url", "bolt://neo4j:7687")\
 .option("authentication.type", "basic")\
 .option("authentication.basic.username", "neo4j")\
 .option("authentication.basic.password", "neo4j")\
 .option("labels", "Person")\
 .load()
display(df)

In [ ]:
df = spark.createDataFrame(
 [(1, "John"),(2, "Thomas")],
 ["id", "name"]
)
df.write.format("org.neo4j.spark.DataSource")\
 .option("url", "bolt://neo4j:7687")\
 .option("authentication.type", "basic")\
 .option("authentication.basic.username", "neo4j")\
 .option("authentication.basic.password", "neo4j")\
 .option("labels", ":Person")\
 .option("node.keys", "id")\
 .mode("Overwrite")\
 .save()